# Classification with Convolutional Neural Net (MNIST)

- 이번 예제부터는 GPU를 이용해 학습하는 과정을 포함합니다.
- MNIST 분류 문제를 CNN을 이용해 풀어보도록 하겠습니다.

In [2]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x58246000 @  0x7ff1c81bc2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [3]:
print(torch.__version__)
print(torch.cuda.is_available())

0.4.1
True


In [0]:
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [0]:
# torch.device()를 통해 gpu를 이용합니다. 
# cuda:뒤의 번호는 gpu를 여러개 동시 사용할 때 쓰이는 gpu 번호입니다.
# gpu 사용이 가능한 경우 torch.cuda.is_available() = True이고 아닌 경우 False가 됩니다.

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
# hyper parameter들을 설정합니다.

num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

## Load data & DataLoader

In [7]:
# MNIST dataset을 불러옵니다.

train_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

Processing...
Done!


In [0]:
# dataloader를 이용해 batch_size만큼 데이터를 불러올 수 있도록 합니다.

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

## Modeling

In [0]:
class ConvNet(nn.Module):
    def __init__(self, num_classes = 10):
        super(ConvNet, self).__init__()
        
        # conv2d에서는 28x28 그대로 나옵니다.
        # pooling을 통해 size는 14x14가 됩니다.
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 5, stride = 1, padding = 2),
            nn.BatchNorm2d(num_features = 16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
            )
        
        # conv2d에서는 14x14 그대로 나옵니다.
        # pooling을 통해 size는 7x7이 됩니다.
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size = 5, stride = 1, padding = 2),
            nn.BatchNorm2d(num_features = 32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
            )
        
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        # flatten
        x = x.reshape(x.size(0), -1)
        out = self.fc(x)
        return out

model = ConvNet(num_classes).to(device)

## Loss & Optimizer

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [12]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1685
Epoch [1/5], Step [200/600], Loss: 0.0488
Epoch [1/5], Step [300/600], Loss: 0.1123
Epoch [1/5], Step [400/600], Loss: 0.1052
Epoch [1/5], Step [500/600], Loss: 0.0435
Epoch [1/5], Step [600/600], Loss: 0.1029
Epoch [2/5], Step [100/600], Loss: 0.0490
Epoch [2/5], Step [200/600], Loss: 0.0592
Epoch [2/5], Step [300/600], Loss: 0.0322
Epoch [2/5], Step [400/600], Loss: 0.0502
Epoch [2/5], Step [500/600], Loss: 0.0910
Epoch [2/5], Step [600/600], Loss: 0.0292
Epoch [3/5], Step [100/600], Loss: 0.0131
Epoch [3/5], Step [200/600], Loss: 0.0829
Epoch [3/5], Step [300/600], Loss: 0.0695
Epoch [3/5], Step [400/600], Loss: 0.0762
Epoch [3/5], Step [500/600], Loss: 0.0217
Epoch [3/5], Step [600/600], Loss: 0.0371
Epoch [4/5], Step [100/600], Loss: 0.0356
Epoch [4/5], Step [200/600], Loss: 0.0304
Epoch [4/5], Step [300/600], Loss: 0.0640
Epoch [4/5], Step [400/600], Loss: 0.0772
Epoch [4/5], Step [500/600], Loss: 0.0036
Epoch [4/5], Step [600/600], Loss:

## Evaluation

In [13]:
# eval() 에서는 dropout, batchnorm 등의 동작이 다릅니다.
# dropout은 작동되지 않고 batchnorm의 running_mean, running_std는 normalization을 위해 사용됩니다.
# 자세한 내용은 torch.nn.BatchNorm2d()을 참조하세요.

model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99.03 %


## Save the model

In [0]:
# weight를 저장합니다.
torch.save(model.state_dict(), 'model.ckpt')